In [ ]:
from datasets import load_dataset
import os
import sys
from langchain_text_splitters import RecursiveCharacterTextSplitter
import dotenv

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

dotenv.load_dotenv()

rag_dataset = load_dataset("neural-bridge/rag-dataset-12000")

In [ ]:
rag_dataset

In [ ]:
len(rag_dataset["train"]["context"][0])
subset = rag_dataset["train"]["context"][:100]

In [ ]:
kit_dir

In [ ]:
text_path = kit_dir + "/data/"
text_file = "subset.txt"

full_path = text_path + text_file
if not os.path.exists(text_path):
    os.makedirs(text_path)

print(full_path)

with open(full_path, "w") as f:
    [f.write("\n\n".join([l for l in subset]))]

In [ ]:
from langchain_community.embeddings.sambanova import SambaStudioEmbeddings
from langchain.vectorstores import Chroma

# # Set the environment variables
# os.environ["SAMBASTUDIO_EMBEDDINGS_BASE_URL"] =
# os.environ["SAMBASTUDIO_EMBEDDINGS_PROJECT_ID"] =
# os.environ["SAMBASTUDIO_EMBEDDINGS_ENDPOINT_ID"] =
# os.environ["SAMBASTUDIO_EMBEDDINGS_API_KEY"] =

from langchain_community.embeddings.sambanova import SambaStudioEmbeddings

embeddings = SambaStudioEmbeddings()

with open(full_path, "r") as f:
    txts = f.read()

# for l in [1000]:

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.create_documents([txts])

# To save
vectorstore = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=kit_dir
    + "/data/"
    + "rag_dataset_12000_train_100samples_1000_100.chromadb",
)

# To load
# vectorstore = Chroma(persist_directory="kit_dir+ "/data/"+ "rag_dataset_12000_train_100samples_1000_100.chromadb", embedding_function=embeddings)